In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
from os.path import isfile
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator, FormatStrFormatter, AutoMinorLocator

In [2]:
def get_url_resp(country="Worldwide"):
    base_url = "https://www.worldometers.info/coronavirus/"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:74.0) Gecko/20100101 Firefox/74.0"}
    if country =="Worldwide":
        url = base_url
    else:
        url =base_url+"country/"+country+"/"
    response = requests.get(url, headers = headers)
    html = response.text
    return BeautifulSoup(html,"html.parser")

def format_case_numbers(soup,country="Worldwide"):
    if country == "Worldwide":
        soup_selector = soup.select(".col-md-6")
        val = [soup_selector[2].find_all(type="text/javascript")[0],soup_selector[3].find_all(type="text/javascript")[0]]
    else:
        soup_selector = soup.find_all("script")
        new_soup =[]
        for i in range(len(soup_selector)):
            if 'categories' in str(soup_selector[i]):
                new_soup.append(soup_selector[i])
        val = new_soup#[:2]
    return val

def get_right_element(val,Status = "Cases"):
    string1 = f"'Total Coronavirus {Status}'"
    new_soup =[]
    for i in range(len(val)):
        if string1 in str(val[i]):
            new_soup.append(val[i])
    return new_soup

def get_both_elements(val):#,country="Worldwide"):
    # if country=="Worldwide":
    #     return val
    # else:
    statuses_tuple = ("Cases","Deaths")
    new_soup = []
    for sta in statuses_tuple:
        new_soup.append(get_right_element(val,sta))
    return new_soup

def get_case_numbers_from_soup(val):#,country="Worldwide",status="Cases"):
    #dates = val.get_text().rsplit("categories",1)[1].split("[")[1].split("]")[0]
    dates = val.get_text().rsplit("categories")[1].split("[")[1].split("]")[0]
    case_num = val.get_text().rsplit("data")[1].split("[")[1].split("]")[0]
    case_num_formatted=case_num.split(',')
    dates_formatted=dates.split(',')
    case_num_lst=[]
    date_lst = []
    # for lst in case_num_lst,date_lst:
    #     lst.append(country)
    #     lst.append(status)

    for i in range(len(case_num_formatted)):
        if case_num_formatted[i] == "null":
            case_num_input = 0
        else:
            case_num_input = int(case_num_formatted[i])
        case_num_lst.append(case_num_input)

    for i in dates_formatted:
        date_lst.append(i.replace('"',''))
    return zip(date_lst,case_num_lst)

def get_case_dictionary(country="Worldwide"):#,status="Cases"):
    soup = get_url_resp(country)
    new_soup1 = format_case_numbers(soup,country)
    val = get_both_elements(new_soup1)
    statuses_tuple = ("Cases","Deaths")
    statuses = {
        "Cases":0,
        "Deaths":1,
    }
    lst=[]
    for sta in statuses_tuple:
        lst.append(dict(get_case_numbers_from_soup(val[statuses[sta]][0])))
    return lst
    #return dict(get_case_numbers_from_soup(val[statuses[status]]))#,country,status)
    # return get_case_numbers_from_soup(val[statuses[status]],country,status)


In [3]:
Countries = ("Worldwide","China","US","Italy","Spain","Germany","France","UK","India","South-Korea","Iran","Netherlands","Belgium","Switzerland","Turkey","Sweden","Indonesia","Portugal")
statuses_tuple = ("Cases","Deaths")
lst=[[],[]]
for country in Countries:
    for sta in statuses_tuple:
        lst[0].append(country)
        lst[1].append(sta)
### WORKS TO HERE ###
# lst = [["Worldwide","US"],["Cases","Cases"]]

country="Worldwide"
country_data=get_case_dictionary(country=country)
print(country_data)
# data=[]
# for country in Countries:
#     country_data=get_case_dictionary(country=country)
#     data.extend(country_data)
# print(data)

[{'Jan 22': 580, 'Jan 23': 845, 'Jan 24': 1317, 'Jan 25': 2015, 'Jan 26': 2800, 'Jan 27': 4581, 'Jan 28': 6058, 'Jan 29': 7813, 'Jan 30': 9823, 'Jan 31': 11950, 'Feb 01': 14553, 'Feb 02': 17391, 'Feb 03': 20630, 'Feb 04': 24545, 'Feb 05': 28266, 'Feb 06': 31439, 'Feb 07': 34876, 'Feb 08': 37552, 'Feb 09': 40553, 'Feb 10': 43099, 'Feb 11': 45134, 'Feb 12': 59287, 'Feb 13': 64438, 'Feb 14': 67100, 'Feb 15': 69197, 'Feb 16': 71329, 'Feb 17': 73332, 'Feb 18': 75184, 'Feb 19': 75700, 'Feb 20': 76677, 'Feb 21': 77673, 'Feb 22': 78651, 'Feb 23': 79205, 'Feb 24': 80087, 'Feb 25': 80828, 'Feb 26': 81820, 'Feb 27': 83112, 'Feb 28': 84615, 'Feb 29': 86604, 'Mar 01': 88585, 'Mar 02': 90443, 'Mar 03': 93016, 'Mar 04': 95314, 'Mar 05': 98425, 'Mar 06': 102050, 'Mar 07': 106099, 'Mar 08': 109991, 'Mar 09': 114381, 'Mar 10': 118948, 'Mar 11': 126214, 'Mar 12': 134509, 'Mar 13': 145416, 'Mar 14': 156475, 'Mar 15': 169517, 'Mar 16': 182414, 'Mar 17': 198159, 'Mar 18': 218744, 'Mar 19': 244902, 'Mar 20':